# **Connect to drive, import libraries, run functions, and define first variables**

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import pickle
import datetime
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import seaborn as sns
import os.path
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import calendar
import joblib
import pickle
import random
from joblib import Parallel, delayed
import multiprocessing
from scipy.stats import ttest_ind
import time

 # Import the ThreadPoolExecutor
from concurrent.futures import ThreadPoolExecutor
import concurrent
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold, GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from scipy.stats import pearsonr
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ExpSineSquared
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

from sklearn.neural_network import MLPRegressor
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Conv1D, MaxPooling1D, LSTM
from sklearn.preprocessing import StandardScaler
from keras.layers import Dropout



# **ML comparison using BRF**

## **preprocessing**

In [ ]:
Total_subset = np.load('/content/drive/MyDrive/PHD/GOES and GPP/Data ready for ML/Data_Aug22_BRFandCMI.npy', allow_pickle=True).item()
Aim = 'GPP_np' # 'GPP_np' and 'RECO_np'
MinSiteSample = 300
shuffle_irt = 10
SAVINGaddress = '/content/drive/MyDrive/Ameriflux_GOES/Gap filling manuscript/Results and Figures/accuracies/'+Aim+'/'
SMF_BRFs = ['BRF1', 'BRF2', 'BRF3', 'BRF5', 'BRF6', 'NIRv', 'NIRvP', 'DSR']
GMF_5BRFs = ['BRF1', 'BRF2', 'BRF3', 'BRF5', 'BRF6', 'NIRv', 'NIRvP', 'DSR']
# General Modeling Features (GMF)
GMF_CMIs = ['CMI_C01','CMI_C02', 'CMI_C03', 'CMI_C04', 'CMI_C05', 'CMI_C06', 'CMI_C07', 'CMI_C08', 'CMI_C09', 'CMI_C10',
                 'CMI_C11', 'CMI_C12', 'CMI_C13', 'CMI_C14', 'CMI_C15', 'CMI_C16']

New_Total_subset = []
tt = 1
maxSamples = 0


## **RF**

In [ ]:
# Lists to store the results
R2test_RF = []
RMSEtest_RF = []
MeanCVR2_RF = []
# Define hyperparameter grid for Random Forest
param_grid = {
    'n_estimators': [100, 250, 500, 800],
    'max_depth': [5, 10, 14, 20, 24],
}

num = 1
for itr in range(shuffle_irt):
  for i in range(len(Total_subset)):
      SiteData = Total_subset[i]
      print(SiteData.shape)

      if MinSiteSample < SiteData.shape[0]:
          # Split into training and testing sets using one-holdout (20% for testing)
          train_data, test_data = train_test_split(SiteData, test_size=0.2, shuffle=True)

          # Separate target variable and predictors
          X_train = train_data[SMF_BRFs]
          y_train = train_data[Aim]
          X_test = test_data[SMF_BRFs]
          y_test = test_data[Aim]

          # Train Random Forest model with grid search and cross-validation
          rf_model = RandomForestRegressor()
          grid_search = GridSearchCV(rf_model, param_grid, cv=4, scoring='r2')
          grid_search.fit(X_train, y_train)

          # Get the best model from grid search and make predictions on test set
          best_rf_model = grid_search.best_estimator_
          y_test_pred = best_rf_model.predict(X_test)

          best_params = grid_search.best_params_
          best_r2_cv = grid_search.best_score_
          print("Best Parameters:", best_params)
          print("Mean R2 from CV:", best_r2_cv)
          MeanCVR2_RF.append(best_r2_cv)

          # Compute R2 and RMSE for testing set
          test_r2 = r2_score(y_test, y_test_pred)
          test_RMSE = np.sqrt(mean_squared_error(y_test, y_test_pred))
          num += 1
          print(num, ' : R2_te =', test_r2, 'RMSE_te =', test_RMSE)

          # Append accuracies to lists
          R2test_RF.append(test_r2)
          RMSEtest_RF.append(test_RMSE)


rf_accuracy = {'R2rf': R2test_RF,
            'RMSErf': RMSEtest_RF,
            'CVR2rf': MeanCVR2_RF}

# Saving the numpy array to a file
file_path_accuracy = SAVINGaddress+'rf_accuracy.npy'
np.save(file_path_accuracy, rf_accuracy)

# Plot the results
plt.boxplot(R2test_RF)
plt.ylim([0, 1])
plt.show()


## **SVR**

In [ ]:
# Lists to store the results
R2test_SVR = []
RMSEtest_SVR = []
MeanCVR2_SVR = []
# Define hyperparameter grid for SVR
param_grid = {
    'C': [0.1, 10, 100, 1000],
    'epsilon': [0.001, 0.01, 0.2],
    'kernel': ['rbf', 'sigmoid', 'poly', 'linear']
}

num = 1
for itr in range(shuffle_irt):
  for i in range(len(Total_subset)):
      SiteData = Total_subset[i]
      print(SiteData.shape)

      if MinSiteSample < SiteData.shape[0]:
          # Split into training and testing sets using one-holdout (20% for testing)
          train_data, test_data = train_test_split(SiteData, test_size=0.2, shuffle=True)

          # Separate target variable and predictors
          X_train = train_data[SMF_BRFs]
          y_train = train_data[Aim]
          X_test = test_data[SMF_BRFs]
          y_test = test_data[Aim]

          # Train SVR model with grid search and cross-validation
          svr_model = SVR(kernel = "rbf")
          grid_search = GridSearchCV(svr_model, param_grid, cv=4, scoring='r2')
          grid_search.fit(X_train, y_train)

          # Get the best model from grid search and its best parameters
          best_svr_model = grid_search.best_estimator_
          best_params = grid_search.best_params_
          best_r2_cv = grid_search.best_score_
          print("Best Parameters:", best_params)
          print("Mean R2 from CV:", best_r2_cv)
          MeanCVR2_SVR.append(best_r2_cv)

          # Make predictions on test set using the best model
          y_test_pred = best_svr_model.predict(X_test)

          # Compute R2 and RMSE for testing set
          test_r2 = r2_score(y_test, y_test_pred)
          test_RMSE = np.sqrt(mean_squared_error(y_test, y_test_pred))
          num += 1
          print(num, ' : R2_te =', test_r2, 'RMSE_te =', test_RMSE)

          # Append accuracies to lists
          R2test_SVR.append(test_r2)
          RMSEtest_SVR.append(test_RMSE)

SVR_accuracy = {'R2SVR': R2test_SVR,
                'RMSESVR': RMSEtest_SVR,
                'CVSVR': MeanCVR2_SVR}

# Saving the numpy array to a file
file_path_accuracy = SAVINGaddress+'SVR_accuracy.npy'
np.save(file_path_accuracy, SVR_accuracy)


# Plot the results
plt.boxplot(R2test_SVR)
plt.ylim([0, 1])
plt.show()


## **MLP**

In [ ]:
# Lists to store the results
R2test_MLP = []
RMSEtest_MLP = []
MeanCVR2_MLP = []
# Define hyperparameter grid for MLP
param_grid = {
    'hidden_layer_sizes': [(50,), (50, 50), (50, 50, 50), (100,), (100, 100), (100, 100, 100), (200,), (200, 200), (200, 200, 200), (300,),
 (300, 300), (300, 300, 300), (500,), (500, 500), (500, 500, 500)],
    'alpha': [0.0001, 0.001, 0.01],
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['adam','sgd'],
    'Alpha': [0.0001, 0.001, 0.01, 0.1]
}

num = 1
for itr in range(shuffle_irt):
  for i in range(len(Total_subset)):
      SiteData = Total_subset[i]
      print(SiteData.shape)

      if MinSiteSample < SiteData.shape[0]:
          # Split into training and testing sets using one-holdout (20% for testing)
          train_data, test_data = train_test_split(SiteData, test_size=0.2, shuffle=True)

          # Separate target variable and predictors
          X_train = train_data[SMF_BRFs]
          y_train = train_data[Aim]
          X_test = test_data[SMF_BRFs]
          y_test = test_data[Aim]

          # Train MLP model with grid search and cross-validation
          mlp_model = MLPRegressor(max_iter=1000)
          grid_search = GridSearchCV(mlp_model, param_grid, cv=4, scoring='r2')
          grid_search.fit(X_train, y_train)

          # Get the best model from grid search and its best parameters
          best_mlp_model = grid_search.best_estimator_
          best_params = grid_search.best_params_
          best_r2_cv = grid_search.best_score_
          print("Best Parameters:", best_params)
          print("Mean R2 from CV:", best_r2_cv)
          MeanCVR2_MLP.append(best_r2_cv)
          # Make predictions on test set using the best model
          y_test_pred = best_mlp_model.predict(X_test)

          # Compute R2 and RMSE for testing set
          test_r2 = r2_score(y_test, y_test_pred)
          test_RMSE = np.sqrt(mean_squared_error(y_test, y_test_pred))
          num += 1
          print(num, ' : R2_te =', test_r2, 'RMSE_te =', test_RMSE)

          # Append accuracies to lists
          R2test_MLP.append(test_r2)
          RMSEtest_MLP.append(test_RMSE)

MLP_accuracy = {'R2MLP': R2test_MLP,
                'RMSEMLP': RMSEtest_MLP,
                'CVMLP': MeanCVR2_MLP}

# Saving the numpy array to a file
file_path_accuracy = SAVINGaddress+'MLP_accuracy.npy'
np.save(file_path_accuracy, MLP_accuracy)

# Plot the results
plt.boxplot(R2test_MLP)
plt.ylim([0, 1])
plt.show()


## **GBR**

In [ ]:
# Lists to store the results
R2test_GBR = []
RMSEtest_GBR = []
MeanCVR2_GBR = []
FI_all = []
# Define hyperparameter grid for GBR
param_grid = {
    'n_estimators': [50, 100, 200, 400],
    'max_depth': [3, 5, 8, 10],
    'learning_rate': [0.01,0.05,0.1]
}

FI_BRFs = []
num = 1
for itr in range(shuffle_irt):
  for i in range(len(Total_subset)):
      SiteData = Total_subset[i]
      print(SiteData.shape)

      if MinSiteSample < SiteData.shape[0]:
          # Split into training and testing sets using one-holdout (20% for testing)
          train_data, test_data = train_test_split(SiteData, test_size=0.2, shuffle=True)

          # Separate target variable and predictors
          X_train = train_data[SMF_BRFs]
          y_train = train_data[Aim]
          X_test = test_data[SMF_BRFs]
          y_test = test_data[Aim]

          # Train GBR model with grid search and cross-validation
          gbr_model = GradientBoostingRegressor()
          grid_search = GridSearchCV(gbr_model, param_grid, cv=4, scoring='r2')
          grid_search.fit(X_train, y_train)

          # Get the best model from grid search and its best parameters
          best_gbr_model = grid_search.best_estimator_
          best_params = grid_search.best_params_
          best_r2_cv = grid_search.best_score_
          print("Best Parameters:", best_params)
          print("Mean R2 from CV:", best_r2_cv)
          MeanCVR2_GBR.append(best_r2_cv)

          # Make predictions on test set using the best model
          y_test_pred = best_gbr_model.predict(X_test)

          # Compute R2 and RMSE for testing set
          test_r2 = r2_score(y_test, y_test_pred)
          test_RMSE = np.sqrt(mean_squared_error(y_test, y_test_pred))
          num += 1
          print(num, ' : R2_te =', test_r2, 'RMSE_te =', test_RMSE)

          R2test_GBR.append(test_r2)
          RMSEtest_GBR.append(test_RMSE)
              # store feature importance
          FI_BRFs = pd.DataFrame()
          FI_BRFs['feature'] = SMF_BRFs

          FI_BRFs['score'] = best_gbr_model.feature_importances_
          FI_all.append(FI_BRFs)

          # Sort the groups by mean score in descending order
          sorted_df = FI_BRFs.sort_values('score', ascending=False)
          # Plotting the feature importance
          plt.figure(figsize=(2, 5))
          plt.barh(sorted_df['feature'], sorted_df['score'], color='black')
          plt.xlabel('Mean Score')
          plt.ylabel('Feature')
          plt.title('Feature Importance')
          plt.show()


# Plot the results
plt.boxplot(R2test_GBR)
plt.ylim([0, 1])
plt.show()

# Concatenate all dataframes in the list into one dataframe
BRFs_df = pd.concat(FI_all)
# Group by 'feature' and calculate the mean score
grouped_df = BRFs_df.groupby('feature')['score'].mean().reset_index()
# Sort the groups by mean score in descending order
sorted_df = grouped_df.sort_values('score', ascending=False)
# Plotting the feature importance
plt.figure(figsize=(2, 1.3))
plt.barh(sorted_df['feature'], sorted_df['score'], color='black')
plt.xlabel('Mean Score')
plt.ylabel('Feature')
plt.title('Feature Importance')
plt.show()


# **CMI and BRF comparison**

In [ ]:
Total_subset = np.load('/content/drive/MyDrive/PHD/GOES and GPP/Data ready for ML/Data_Aug17_BRFandCMI.npy', allow_pickle=True).item()

Aims = ['RECO_np', 'GPP_np'] # 'GPP_np' and 'RECO_np'
MinSiteSample = 100
CMI_18b = ['CMI_C01','CMI_C02', 'CMI_C03', 'CMI_C04', 'CMI_C05', 'CMI_C06', 'CMI_C07', 'CMI_C08', 'CMI_C09', 'CMI_C10',
          'CMI_C11', 'CMI_C12', 'CMI_C13', 'CMI_C14', 'CMI_C15', 'CMI_C16']
BRF_6b = ['BRF1', 'BRF2', 'BRF3', 'BRF5', 'BRF6']
Com_f = CMI_18b + BRF_6b

data = pd.concat(Total_subset)

print(len(Total_subset))
print(len(Total_DataCMI))

def train_and_test_model(data, features, target):
    train_data, test_data = train_test_split(data, test_size=0.2, shuffle=True)

    X_train = train_data[features]
    y_train = train_data[target]
    X_test = test_data[features]
    y_test = test_data[target]

    param_grid = {
        'n_estimators': [50, 100, 200, 400],
        'max_depth': [3, 5, 8, 10],
        'learning_rate': [0.01,0.05,0.1]
    }

    # Train GBR model with grid search and cross-validation
    gbr_model = GradientBoostingRegressor()
    grid_search = GridSearchCV(gbr_model, param_grid, cv=4, scoring='r2')
    grid_search.fit(X_train, y_train)

    model = grid_search.best_estimator_

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_r2 = r2_score(y_train, y_train_pred)
    train_RMSE = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_r2 = r2_score(y_test, y_test_pred)
    test_RMSE = np.sqrt(mean_squared_error(y_test, y_test_pred))
    print(train_r2, train_RMSE, test_r2, test_RMSE)

    return train_r2, train_RMSE, test_r2, test_RMSE, model.feature_importances_

R2_BRF_RECO, RMSE_BRF_RECO, R2_CMI_RECO, RMSE_CMI_RECO, R2_Com_RECO, RMSE_Com_RECO  = [], [], [], [], [], []
R2_BRF_GPP, RMSE_BRF_GPP, R2_CMI_GPP, RMSE_CMI_GPP, R2_Com_GPP, RMSE_Com_GPP = [], [], [], [], [], []
all_FI_BRFs, all_FI_CMIs, all_FI_Coms = [], [], []

for Aim in Aims:
    R2_BRF, RMSE_BRF, R2_CMI, RMSE_CMI , R2_Com, RMSE_Com = [], [], [], [], [], []

    for Isite in range(101):
        print(SiteData)
        selected_data = Total_subset[Isite]

        if MinSiteSample < selected_data.shape[0]:

            r2_BRF_tr, rmse_BRF_tr, r2_BRF_te, rmse_BRF_te, FI_BRF = train_and_test_model(selected_data, BRF_6b, Aim)
            r2_CMI_tr, rmse_CMI_tr, r2_CMI_te, rmse_CMI_te, FI_CMI = train_and_test_model(selected_data, CMI_18b, Aim)
            r2_Com_tr, rmse_Com_tr, r2_Com_te, rmse_Com_te, FI_Com = train_and_test_model(selected_data, Com_f, Aim)

            R2_BRF.append(r2_BRF_te)
            RMSE_BRF.append(rmse_BRF_te)

            R2_CMI.append(r2_CMI_te)
            RMSE_CMI.append(rmse_CMI_te)

            R2_Com.append(r2_Com_te)
            RMSE_Com.append(rmse_Com_te)

        # store feature importance
        FI_BRFs = pd.DataFrame()
        FI_BRFs['feature'] = BRF_6b
        FI_BRFs['score'] = FI_BRF
        all_FI_BRFs.append(FI_BRFs)

        FI_CMIs = pd.DataFrame()
        FI_CMIs['feature'] = CMI_18b
        FI_CMIs['score'] = FI_CMI
        all_FI_CMIs.append(FI_CMIs)

        FI_Coms = pd.DataFrame()
        FI_Coms['feature'] = Com_f
        FI_Coms['score'] = FI_Com
        all_FI_Coms.append(FI_Coms)

    if Aim == 'RECO_np':
        R2_BRF_RECO = R2_BRF
        RMSE_BRF_RECO = RMSE_BRF
        R2_CMI_RECO = R2_CMI
        RMSE_CMI_RECO = RMSE_CMI
        R2_Com_RECO = R2_Com
        RMSE_Com_RECO = RMSE_Com

    if Aim == 'GPP_np':
        R2_BRF_GPP = R2_BRF
        RMSE_BRF_GPP = RMSE_BRF
        R2_CMI_GPP = R2_CMI
        RMSE_CMI_GPP = RMSE_CMI
        R2_Com_GPP = R2_Com
        RMSE_Com_GPP = RMSE_Com


In [ ]:
SAVINGaddress = '/content/drive/MyDrive/Ameriflux_GOES/Gap filling manuscript/Results and Figures/accuracies/BRF_CMI/'

accuracy = {"R2_BRF_RECO": R2_BRF_RECO,
    "RMSE_BRF_RECO": RMSE_BRF_RECO,
    "R2_CMI_RECO": R2_CMI_RECO,
    "RMSE_CMI_RECO": RMSE_CMI_RECO,
    "R2_Com_RECO": R2_Com_RECO,
    "RMSE_Com_RECO": RMSE_Com_RECO,
    "R2_BRF_GPP": R2_BRF_GPP,
    "RMSE_BRF_GPP": RMSE_BRF_GPP,
    "R2_CMI_GPP": R2_CMI_GPP,
    "RMSE_CMI_GPP": RMSE_CMI_GPP,
    "R2_Com_GPP": R2_Com_GPP,
    "RMSE_Com_GPP": RMSE_Com_GPP}

# Saving the numpy array to a file
file_path_accuracy = SAVINGaddress+'accuracyBRF_CMI.npy'
np.save(file_path_accuracy, accuracy)


# Calculate mean and standard deviation for each list
listsR2 = [R2_BRF_RECO, R2_CMI_RECO, R2_Com_RECO, R2_BRF_GPP, R2_CMI_GPP, R2_Com_GPP]
listsRMSE = [RMSE_BRF_RECO, RMSE_CMI_RECO, RMSE_Com_RECO, RMSE_BRF_GPP, RMSE_CMI_GPP, RMSE_Com_GPP]

# Replace the original values in the rf_accuracy dictionary with the filtered lists
accuracy1 = {
    "R2_BRF_RECO": listsR2[0],
    "RMSE_BRF_RECO": listsRMSE[0],
    "R2_CMI_RECO": listsR2[1],
    "RMSE_CMI_RECO": listsRMSE[1],
    "R2_Com_RECO": listsR2[2],
    "RMSE_Com_RECO": listsRMSE[2],
    "R2_BRF_GPP": listsR2[3],
    "RMSE_BRF_GPP": listsRMSE[3],
    "R2_CMI_GPP": listsR2[4],
    "RMSE_CMI_GPP": listsRMSE[4],
    "R2_Com_GPP": listsR2[5],
    "RMSE_Com_GPP": listsRMSE[5]
}

# Saving the numpy array to a file
file_path_accuracy = SAVINGaddress+'accuracyBRF_CMI_V1.npy'
np.save(file_path_accuracy, accuracy1)

FI_save = { "all_FI_BRFs": all_FI_BRFs,
    "all_FI_CMIs": all_FI_CMIs,
    "all_FI_Coms": all_FI_Coms}

# Saving the numpy array to a file
file_path_accuracy = SAVINGaddress+'FI_save.npy'
np.save(file_path_accuracy, FI_save)



In [ ]:
def create_box_plots(ax, data1, data2, data3, labels, ylabel, legend_labels, box_colors, box_hatch):
    bp1 = ax.boxplot(data1, positions=[1, 4], labels=labels, widths=0.3, patch_artist=True)
    bp2 = ax.boxplot(data2, positions=[1.7, 4.7], labels=labels, widths=0.3, patch_artist=True)
    bp3 = ax.boxplot(data3, positions=[2.4, 5.4], labels=labels, widths=0.3, patch_artist=True)

    for bp in [bp1, bp2, bp3]:
        for box in bp['boxes']:
            box.set(color='black', linewidth=1)
            if bp == bp1:
                box.set(facecolor=box_colors)
            elif bp == bp2:
                box.set(facecolor='white')
            else :
                box.set(hatch=box_hatch)
                box.set(facecolor='white')

    legend_patches = [mpatches.Patch(color=box_colors, label=legend_labels[0]),
                      mpatches.Patch(facecolor='white', edgecolor='black', label=legend_labels[1]),
                      mpatches.Patch(hatch=box_hatch, facecolor='white', edgecolor='black', label=legend_labels[2])]

    ax.set_ylabel(ylabel)
    ax.legend(handles=legend_patches)
    ax.set_xticks([1.7, 4.7])
    ax.set_xticklabels(labels)

# Define the path to the saved Numpy file
file_path_accuracy = '/content/drive/MyDrive/Ameriflux_GOES/Gap filling manuscript/Results and Figures/accuracies/BRF_CMI/accuracyBRF_CMI_V1.npy'

# Load the dictionary from the Numpy file
loaded_accuracy = np.load(file_path_accuracy, allow_pickle=True).item()

# Extract the values from the loaded dictionary
R2_BRF_RECO = loaded_accuracy["R2_BRF_RECO"]
RMSE_BRF_RECO = loaded_accuracy["RMSE_BRF_RECO"]
R2_CMI_RECO = loaded_accuracy["R2_CMI_RECO"]
RMSE_CMI_RECO = loaded_accuracy["RMSE_CMI_RECO"]
R2_Com_RECO = loaded_accuracy["R2_Com_RECO"]
RMSE_Com_RECO = loaded_accuracy["RMSE_Com_RECO"]
R2_BRF_GPP = loaded_accuracy["R2_BRF_GPP"]
RMSE_BRF_GPP = loaded_accuracy["RMSE_BRF_GPP"]
R2_CMI_GPP = loaded_accuracy["R2_CMI_GPP"]
RMSE_CMI_GPP = loaded_accuracy["RMSE_CMI_GPP"]
R2_Com_GPP = loaded_accuracy["R2_Com_GPP"]
RMSE_Com_GPP = loaded_accuracy["RMSE_Com_GPP"]

# Set up the figure and the axes
fig, axes = plt.subplots(2, 1, figsize=(3, 6))

labels = ['GPP', 'RECO']
box_colors_GPP = 'black'
box_hatch_RECO = '//'
legend_labels = ['BRF', 'CMI', 'BRF+CMI']

# Create the box plots for R2
create_box_plots(axes[0], [R2_BRF_GPP, R2_BRF_RECO], [R2_CMI_GPP, R2_CMI_RECO], [R2_Com_GPP, R2_Com_RECO], labels, '$R^2$', legend_labels, box_colors_GPP, box_hatch_RECO)
# Create the box plots for RMSE
create_box_plots(axes[1], [RMSE_BRF_GPP, RMSE_BRF_RECO], [RMSE_CMI_GPP, RMSE_CMI_RECO], [RMSE_Com_GPP, RMSE_Com_RECO], labels, 'RMSE (µmol $CO_2$ $m^{-2}$ $s^{-1}$)', legend_labels, box_colors_GPP, box_hatch_RECO)
axes[0].set_ylim([0, 1])
axes[0].legend()
plt.tight_layout()
plt.show()


In [ ]:
# Concatenate all dataframes in the list into one dataframe
BRFs_df = pd.concat(all_FI_BRFs)
# Group by 'feature' and calculate the mean score
grouped_df = BRFs_df.groupby('feature')['score'].mean().reset_index()
# Sort the groups by mean score in descending order
sorted_df = grouped_df.sort_values('score', ascending=False)
# Plotting the feature importance
plt.figure(figsize=(2, 1.3))
plt.barh(sorted_df['feature'], sorted_df['score'], color='black')
plt.xlabel('Mean Score')
plt.ylabel('Feature')
plt.title('Feature Importance')
plt.show()

# Concatenate all dataframes in the list into one dataframe
CMIs_df = pd.concat(all_FI_CMIs)
# Group by 'feature' and calculate the mean score
grouped_df = CMIs_df.groupby('feature')['score'].mean().reset_index()
# Sort the groups by mean score in descending order
sorted_df = grouped_df.sort_values('score', ascending=False)
# Plotting the feature importance
plt.figure(figsize=(2, 4))
plt.barh(sorted_df['feature'], sorted_df['score'], color='black')
plt.xlabel('Mean Score')
plt.ylabel('Feature')
plt.title('Feature Importance')
plt.show()

# Concatenate all dataframes in the list into one dataframe
Coms_df = pd.concat(all_FI_Coms)
# Group by 'feature' and calculate the mean score
grouped_df = Coms_df.groupby('feature')['score'].mean().reset_index()
# Sort the groups by mean score in descending order
sorted_df = grouped_df.sort_values('score', ascending=False)
# Plotting the feature importance
plt.figure(figsize=(2, 5))
plt.barh(sorted_df['feature'], sorted_df['score'], color='black')
plt.xlabel('Mean Score')
plt.ylabel('Feature')
plt.title('Feature Importance')
plt.show()



# **Different Gap size**

## four years training

### preprocessing and functions

In [ ]:
Total_subset = np.load('/content/drive/MyDrive/PHD/GOES and GPP/Data ready for ML/Subset_Jun21.npy', allow_pickle=True).item()
MinSiteSample = 2000
Sizes = []
AllData = []
# Create lists to store the accuracies and scores
All_traindata = []
All_testdata = []
num = 1
Aim = "RECO_np" # "GPP_np", "RECO_np"



In [ ]:
def split_data(df, test_scenario):
    df = df.sample(frac=1).reset_index(drop=True)
    if test_scenario == 1:
        # Generate test_days with groups having at least two samples
        df_idx = df.reset_index(drop=True)
        grouped_df = df_idx.groupby('y_doy')
        valid_groups = grouped_df.filter(lambda x: len(x) > 2)
        test_days = np.random.choice(valid_groups['y_doy'].unique(), size=4000, replace=True)
        # Filter test_data and ensure each group has at least two samples
        test_data = valid_groups[valid_groups['y_doy'].isin(test_days)].groupby('y_doy').apply(lambda x: x.sample(n=2)).copy()

        # Filter train_data based on the test_data indices
        train_data = df_idx.drop(test_data.index.get_level_values(None))
        print("train_data", train_data.shape, "test_data", test_data.shape)

    elif test_scenario == 2:
        # Generate test_days with groups having at least two samples
        df_idx = df.reset_index(drop=True)
        grouped_df = df_idx.groupby('y_doy')
        valid_groups = grouped_df.filter(lambda x: len(x) > 3)
        test_days = np.random.choice(valid_groups['y_doy'].unique(), size=2000, replace=True)
        # Filter test_data and ensure each group has at least two samples
        test_data = valid_groups[valid_groups['y_doy'].isin(test_days)].groupby('y_doy').apply(lambda x: x.sample(n=3)).copy()
        # Filter train_data based on the test_data indices
        train_data = df_idx.drop(test_data.index.get_level_values(None))
        print("train_data", train_data.shape, "test_data", test_data.shape)

    elif test_scenario == 3:
        # Generate test_days with groups having at least four samples
        df_idx = df.reset_index(drop=True)
        grouped_df = df_idx.groupby('y_doy')
        valid_groups = grouped_df.filter(lambda x: len(x) > 4)
        test_days = np.random.choice(valid_groups['y_doy'].unique(), size=1200, replace=True)
        # Filter test_data and ensure each group has at least four samples
        test_data = valid_groups[valid_groups['y_doy'].isin(test_days)].groupby('y_doy').apply(lambda x: x.sample(n=4)).copy()
        # Filter train_data based on the test_data indices
        train_data = df_idx.drop(test_data.index.get_level_values(None))
        print("train_data", train_data.shape, "test_data", test_data.shape)

    elif test_scenario == 4:
        # Generate test_days with groups having at least seven samples
        df_idx = df.reset_index(drop=True)
        grouped_df = df_idx.groupby('y_doy')
        valid_groups = grouped_df.filter(lambda x: len(x) > 8)
        test_days = np.random.choice(valid_groups['y_doy'].unique(), size=500, replace=True)
        # Filter test_data and ensure each group has at least seven samples
        test_data = valid_groups[valid_groups['y_doy'].isin(test_days)].groupby('y_doy').apply(lambda x: x.sample(n=8)).copy()
        # Filter train_data based on the test_data indices
        train_data = df_idx.drop(test_data.index.get_level_values(None))
        print("train_data", train_data.shape, "test_data", test_data.shape)

    elif test_scenario == 5:
        # Generate test_days with groups having at least ten samples
        df_idx = df.reset_index(drop=True)
        grouped_df = df_idx.groupby('y_doy')
        valid_groups = grouped_df.filter(lambda x: len(x) >= 8)
        test_days = np.random.choice(valid_groups['y_doy'].unique(), size=500, replace=True)
        # Filter test_data and ensure each group has at least ten samples
        test_data = valid_groups[valid_groups['y_doy'].isin(test_days)].groupby('y_doy').apply(lambda x: x.sample(n=8)).copy()
        # Filter train_data based on the test_data indices
        train_data = df_idx.drop(test_data.index.get_level_values(None))
        print("train_data", train_data.shape, "test_data", test_data.shape)

    elif test_scenario == 6:
        # Generate test_days with groups having at least one sample
        df_idx = df.reset_index(drop=True)
        grouped_df = df_idx.groupby('y_doy')
        valid_groups = grouped_df.filter(lambda x: len(x) >= 8)
        test_days = np.random.choice(valid_groups['y_doy'].unique(), size=57, replace=True)
        # Include one day before and after the selected y_doy
        test_days_extended = []
        for day in test_days:
            test_days_extended.extend([day-1, day, day+1, day+2])
        test_days_extended = np.array(test_days_extended)
        # Filter test_data and ensure each group has all samples for the selected y_doy and the adjacent days
        test_data = valid_groups[valid_groups['y_doy'].isin(test_days_extended)].copy()
        # Filter train_data based on the test_data indices
        train_data = df_idx.drop(test_data.index)
        print("train_data", train_data.shape, "test_data", test_data.shape)

    elif test_scenario == 7:
        # Generate test_days with groups having at least one sample
        df_idx = df.reset_index(drop=True)
        grouped_df = df_idx.groupby('y_doy')
        valid_groups = grouped_df.filter(lambda x: len(x) >= 8)
        test_days = np.random.choice(valid_groups['y_doy'].unique(), size=30, replace=True)
        # Include two days before and after the selected y_doy
        test_days_extended = []
        for day in test_days:
            test_days_extended.extend([day-3, day-2, day-1, day, day+1, day+2, day+3])
        test_days_extended = np.array(test_days_extended)
        # Filter test_data and ensure each group has all samples for the selected y_doy and the adjacent days
        test_data = valid_groups[valid_groups['y_doy'].isin(test_days_extended)].copy()
        # Filter train_data based on the test_data indices
        train_data = df_idx.drop(test_data.index)
        print("train_data", train_data.shape, "test_data", test_data.shape)

    elif test_scenario == 8:
        # Generate test_days with groups having at least one sample
        df_idx = df.reset_index(drop=True)
        grouped_df = df_idx.groupby('y_doy')
        valid_groups = grouped_df.filter(lambda x: len(x) >= 8)
        test_days = np.random.choice(valid_groups['y_doy'].unique(), size=17, replace=True)
        # Include three days before and after the selected y_doy
        test_days_extended = []
        for day in test_days:
            test_days_extended.extend([day-7, day-6, day-5, day-4, day-3, day-2, day-1, day,
                                       day+1, day+2, day+3, day+4, day+5, day+6])
        test_days_extended = np.array(test_days_extended)
        # Filter test_data and ensure each group has all samples for the selected y_doy and the adjacent days
        test_data = valid_groups[valid_groups['y_doy'].isin(test_days_extended)].copy()
        # Filter train_data based on the test_data indices
        train_data = df_idx.drop(test_data.index)
        print("train_data", train_data.shape, "test_data", test_data.shape)

    elif test_scenario == 9:
        # Generate test_days with groups having at least one sample
        df_idx = df.reset_index(drop=True)
        grouped_df = df_idx.groupby('y_month')
        valid_groups = grouped_df.filter(lambda x: len(x) >= 250)
        test_months = np.random.choice(valid_groups['y_month'].unique(), size=7, replace=True)
        # Filter test_data and ensure each group has all samples for the selected y_doy and the adjacent days
        test_data = valid_groups[valid_groups['y_month'].isin(test_months)].copy()
        # Filter train_data based on the test_data indices
        train_data = df_idx.drop(test_data.index)
        print("train_data", train_data.shape, "test_data", test_data.shape)

    elif test_scenario == 10:
        # Generate test_days with groups having at least one sample
        df_idx = df.reset_index(drop=True)
        grouped_df = df_idx.groupby('y_month')
        valid_groups = grouped_df.filter(lambda x: len(x) >= 250)
        test_months = np.random.choice(valid_groups['y_month'].unique(), size=4, replace=True)
        # Include ten days before and after the selected DOY
        test_months_extended = []
        for Month in test_months:
            test_months_extended.extend([Month, Month+1])
        test_months_extended = np.array(test_months_extended)
        # Filter test_data and ensure each group has all samples for the selected DOY and the adjacent days
        test_data = valid_groups[valid_groups['y_month'].isin(test_months_extended)].copy()
        # Filter train_data based on the test_data indices
        train_data = df_idx.drop(test_data.index)
        print("train_data", train_data.shape, "test_data", test_data.shape)

    elif test_scenario == 11:
        # Generate test_days with groups having at least one sample
        df_idx = df.reset_index(drop=True)
        grouped_df = df_idx.groupby('y_month')
        valid_groups = grouped_df.filter(lambda x: len(x) >= 250)
        test_months = np.random.choice(valid_groups['y_month'].unique(), size=3, replace=True)
        # Include ten days before and after the selected DOY
        test_months_extended = []
        for Month in test_months:
            test_months_extended.extend([Month-1, Month, Month+1])
        test_months_extended = np.array(test_months_extended)
        # Filter test_data and ensure each group has all samples for the selected DOY and the adjacent days
        test_data = valid_groups[valid_groups['y_month'].isin(test_months_extended)].copy()
        # Filter train_data based on the test_data indices
        train_data = df_idx.drop(test_data.index)
        print("train_data", train_data.shape, "test_data", test_data.shape)

    elif test_scenario == 12:
        # Generate test_days with groups having at least one sample
        df_idx = df.reset_index(drop=True)
        grouped_df = df_idx.groupby('y_month')
        valid_groups = grouped_df.filter(lambda x: len(x) >= 250)
        test_months = np.random.choice(valid_groups['y_month'].unique(), size=2, replace=True)
        # Include ten days before and after the selected DOY
        test_months_extended = []
        for Month in test_months:
            test_months_extended.extend([Month-1, Month, Month+1, Month+2])
        test_months_extended = np.array(test_months_extended)
        # Filter test_data and ensure each group has all samples for the selected DOY and the adjacent days
        test_data = valid_groups[valid_groups['y_month'].isin(test_months_extended)].copy()
        # Filter train_data based on the test_data indices
        train_data = df_idx.drop(test_data.index)
        print("train_data", train_data.shape, "test_data", test_data.shape)

    elif test_scenario == 13:
        # Generate test_days with groups having at least one sample
        df_idx = df.reset_index(drop=True)
        grouped_df = df_idx.groupby('y_month')
        valid_groups = grouped_df.filter(lambda x: len(x) >= 250)
        test_months = np.random.choice(valid_groups['y_month'].unique(), size=2, replace=True)
        # Include ten days before and after the selected DOY
        test_months_extended = []
        for Month in test_months:
            test_months_extended.extend([Month-2, Month-1, Month, Month+1, Month+2])
        test_months_extended = np.array(test_months_extended)
        # Filter test_data and ensure each group has all samples for the selected DOY and the adjacent days
        test_data = valid_groups[valid_groups['y_month'].isin(test_months_extended)].copy()
        # Filter train_data based on the test_data indices
        train_data = df_idx.drop(test_data.index)
        print("train_data", train_data.shape, "test_data", test_data.shape)

    elif test_scenario == 14:
        # Generate test_days with groups having at least one sample
        df_idx = df.reset_index(drop=True)
        grouped_df = df_idx.groupby('y_month')
        valid_groups = grouped_df.filter(lambda x: len(x) >= 200)
        test_months = np.random.choice(valid_groups['y_month'].unique(), size=2, replace=True)
        # Include ten days before and after the selected DOY
        test_months_extended = []
        for Month in test_months:
            test_months_extended.extend([Month-2, Month-1, Month, Month+1, Month+2, Month+3])
        test_months_extended = np.array(test_months_extended)
        # Filter test_data and ensure each group has all samples for the selected DOY and the adjacent days
        test_data = valid_groups[valid_groups['y_month'].isin(test_months_extended)].copy()
        # Filter train_data based on the test_data indices
        train_data = df_idx.drop(test_data.index)
        print("train_data", train_data.shape, "test_data", test_data.shape)

    elif test_scenario == 15:
        # Generate test_days with groups having at least one sample
        df_idx = df.reset_index(drop=True)
        grouped_df = df_idx.groupby('Year')
        valid_groups = grouped_df.filter(lambda x: len(x) >= 4000)
        test_year = np.random.choice(valid_groups['Year'].unique(), size=1, replace=True)
        # Filter test_data and ensure each group has all samples for the selected DOY and the adjacent days
        test_data = valid_groups[valid_groups['Year'].isin(test_year)].copy()
        # Filter train_data based on the test_data indices
        train_data = df_idx.drop(test_data.index)
        print("train_data", train_data.shape, "test_data", test_data.shape)
    else:
        raise ValueError("Invalid test_scenario value. Choose a value between 1 and 15.")

    return train_data, test_data

# Define a function to train and evaluate the RF regression model
def train_and_evaluate_model(train_data, test_data, Features):
    print(Aim)
    X_train = train_data[Features]
    y_train = train_data[Aim]
    X_test = test_data[Features]
    y_test = test_data[Aim]
    param_grid = {
        'n_estimators': [50, 100, 200, 400],
        'max_depth': [3, 5, 8, 10],
        'learning_rate': [0.01,0.05,0.1]
    }
    # Train GBR model with grid search and cross-validation
    gbr_model = GradientBoostingRegressor()
    grid_search = GridSearchCV(gbr_model, param_grid, cv=4, scoring='r2')
    grid_search.fit(X_train, y_train)
    model = grid_search.best_estimator_
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    return rmse, r2

print('done')



done
